In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather
import glob
import os
import pyarrow
from xclim import ensembles as ens
from xclim import subset

In [2]:
import matplotlib.pyplot as plt

In [3]:
folder = "/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/"
out_folder = "/home/mlopez/EXEC/TimeMeansModels/"

# Select variable

In [4]:
#Variable 
var="tg_mean"
#var ="tn_mean"
#var ="tx_mean"
#var ="solidprcptot"
#var ="prcptot"
#var ="dlyfrzthw"
#var ="growing_season_length"
#var ="DJC"


files85 = glob.glob(folder+"*"+"rcp85_"+var+"_annual.nc")
files45 = glob.glob(folder+"*"+"rcp45_"+var+"_annual.nc")


filesSeas45 =  glob.glob(folder+"*rcp45_"+var+"_seasonal.nc")
filesSeas85 =  glob.glob(folder+"*rcp85_"+var+"_seasonal.nc")



# FOR RCP 45

In [5]:
dsEns45= ens.create_ensemble(files45)
#dsEns45

## Getting time windows and percentiles

In [6]:
# Table 
# Mapping data - 30 year means
tmp1 = dsEns45.sel(time=(dsEns45.time.dt.year>=1981))
window = 30
time1 = tmp1.time[0::window]
# 30 y means
ds30yavgTable = tmp1.coarsen(time=window).mean()
ds30yavgTable['time'] = time1
# spatial mean
perc30yavgTable = ens.ensemble_percentiles(ds30yavgTable)
#perc30yavgTable 

In [7]:
%time dfperT45 = perc30yavgTable.drop('realization').to_dataframe().dropna()

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)



CPU times: user 54.3 s, sys: 3min 39s, total: 4min 33s
Wall time: 5.18 s


In [8]:
dfperT45['season'] = "annual"

In [9]:
dfperT45

prcptot_p10  prcptot_p50  prcptot_p90  season
lat       lon        time                                                     
66.623306 -89.045212 1981-01-01   242.776947   248.862839   253.129547  annual
                     2011-01-01   250.770477   258.080261   273.385803  annual
                     2041-01-01   268.117340   281.025696   298.878113  annual
                     2071-01-01   284.791718   298.072601   314.719666  annual
          -88.961884 1981-01-01   240.618851   246.484695   250.704636  annual
...                                      ...          ...          ...     ...
40.041039 -74.129143 2071-01-01  1217.491821  1282.380371  1355.752563  annual
          -74.045807 1981-01-01  1175.379761  1226.125488  1238.832764  annual
                     2011-01-01  1189.914307  1228.283569  1276.126465  annual
                     2041-01-01  1209.976929  1288.678589  1329.441040  annual
                     2071-01-01  1214.395996  1276.816772  1354.748291  annual

[304824 rows x 4 columns]

# FOR RCP85

In [10]:
dsEns85= ens.create_ensemble(files85)
#dsEns85

## Getting time windows and percentiles

In [11]:
#for (file, percentile, season) in ((files45, 45, "annual"), (files85, 85, "annual"),
#                                  (filesSeas45, 45, )):
# Table 
# Mapping data - 30 year means
tmp1 = dsEns85.sel(time=(dsEns85.time.dt.year>=1981))
window = 30
time1 = tmp1.time[0::window]
# 30 y means
ds30yavgTable = tmp1.coarsen(time=window).mean()
ds30yavgTable['time'] = time1
# spatial mean
perc30yavgTable85 = ens.ensemble_percentiles(ds30yavgTable)
#perc30yavgTable 

%time dfperT85 = perc30yavgTable85.drop('realization').to_dataframe().dropna()
dfperT85['season'] = "annual"

dfperT85

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)



CPU times: user 54 s, sys: 4min 48s, total: 5min 42s
Wall time: 6.98 s


prcptot_p10  prcptot_p50  prcptot_p90  season
lat       lon        time                                                     
66.623306 -89.045212 1981-01-01   245.039719   248.596481   258.564270  annual
                     2011-01-01   260.379547   266.665070   274.814545  annual
                     2041-01-01   276.355499   296.327087   314.078552  annual
                     2071-01-01   296.454498   320.437500   370.024689  annual
          -88.961884 1981-01-01   242.724014   246.173447   256.140045  annual
...                                      ...          ...          ...     ...
40.041039 -74.129143 2071-01-01  1252.603271  1315.043823  1396.519409  annual
          -74.045807 1981-01-01  1189.647095  1231.080933  1246.812256  annual
                     2011-01-01  1202.679321  1257.817017  1306.370361  annual
                     2041-01-01  1213.784058  1263.579712  1321.775024  annual
                     2071-01-01  1245.081543  1309.232788  1389.138306  annual

[304824 rows x 4 columns]

## TODO
- Cut by region
- get the mean by region
- Do the same for rcp84 and seasonal 45 and 85. 
- Put all together into the table

# FOR Seasonal RCP45

## Loop to create all 4 dataframes

In [12]:
dsEns45= ens.create_ensemble(files45)
dsEns45.time.dt.season[0:4]
# Seasonal example 30 y means
listds45 = []
i=0
for s in dsEns45.time.dt.season[0:4]:
    print (i)
    tmp1 = dsEns45.sel(time=(dsEns45.time.dt.year>=1981))
    tmp1 = tmp1.sel(time=(tmp1.time.dt.season==s))
    window = 30
    time1 = tmp1.time[0::window]
    # 30 y means
    ds30yavgTable = tmp1.coarsen(time=window).mean()
    ds30yavgTable['time'] = time1
    # percentiles
    perc30yavgTable45 = ens.ensemble_percentiles(ds30yavgTable)
    #perc30yavgTable 
    dfSeas45 = perc30yavgTable45.drop('realization').to_dataframe().dropna()
    if i== 0: season = "winter" 
    elif i == 1: season = "spring"
    elif i == 2: season = "summer"
    elif i == 3: season = "fall"
    print (season)
    dfSeas45["season"] = season
    print(dfSeas45)
    i = i+1
    listds45.append(dfSeas45)

0


  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)



winter
                                 prcptot_p10  prcptot_p50  prcptot_p90  season
lat       lon        time                                                     
66.623306 -89.045212 1981-01-01   242.776947   248.862839   253.129547  winter
                     2011-01-01   250.770477   258.080261   273.385803  winter
                     2041-01-01   268.117340   281.025696   298.878113  winter
                     2071-01-01   284.791718   298.072601   314.719666  winter
          -88.961884 1981-01-01   240.618851   246.484695   250.704636  winter
...                                      ...          ...          ...     ...
40.041039 -74.129143 2071-01-01  1217.491821  1282.380371  1355.752563  winter
          -74.045807 1981-01-01  1175.379761  1226.125488  1238.832764  winter
                     2011-01-01  1189.914307  1228.283569  1276.126465  winter
                     2041-01-01  1209.976929  1288.678589  1329.441040  winter
                     2071-01-01  1214.395996 

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)



spring
                                 prcptot_p10  prcptot_p50  prcptot_p90  season
lat       lon        time                                                     
66.623306 -89.045212 1981-01-01   242.776947   248.862839   253.129547  spring
                     2011-01-01   250.770477   258.080261   273.385803  spring
                     2041-01-01   268.117340   281.025696   298.878113  spring
                     2071-01-01   284.791718   298.072601   314.719666  spring
          -88.961884 1981-01-01   240.618851   246.484695   250.704636  spring
...                                      ...          ...          ...     ...
40.041039 -74.129143 2071-01-01  1217.491821  1282.380371  1355.752563  spring
          -74.045807 1981-01-01  1175.379761  1226.125488  1238.832764  spring
                     2011-01-01  1189.914307  1228.283569  1276.126465  spring
                     2041-01-01  1209.976929  1288.678589  1329.441040  spring
                     2071-01-01  1214.395996 

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)



summer
                                 prcptot_p10  prcptot_p50  prcptot_p90  season
lat       lon        time                                                     
66.623306 -89.045212 1981-01-01   242.776947   248.862839   253.129547  summer
                     2011-01-01   250.770477   258.080261   273.385803  summer
                     2041-01-01   268.117340   281.025696   298.878113  summer
                     2071-01-01   284.791718   298.072601   314.719666  summer
          -88.961884 1981-01-01   240.618851   246.484695   250.704636  summer
...                                      ...          ...          ...     ...
40.041039 -74.129143 2071-01-01  1217.491821  1282.380371  1355.752563  summer
          -74.045807 1981-01-01  1175.379761  1226.125488  1238.832764  summer
                     2011-01-01  1189.914307  1228.283569  1276.126465  summer
                     2041-01-01  1209.976929  1288.678589  1329.441040  summer
                     2071-01-01  1214.395996 

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)



fall
                                 prcptot_p10  prcptot_p50  prcptot_p90 season
lat       lon        time                                                    
66.623306 -89.045212 1981-01-01   242.776947   248.862839   253.129547   fall
                     2011-01-01   250.770477   258.080261   273.385803   fall
                     2041-01-01   268.117340   281.025696   298.878113   fall
                     2071-01-01   284.791718   298.072601   314.719666   fall
          -88.961884 1981-01-01   240.618851   246.484695   250.704636   fall
...                                      ...          ...          ...    ...
40.041039 -74.129143 2071-01-01  1217.491821  1282.380371  1355.752563   fall
          -74.045807 1981-01-01  1175.379761  1226.125488  1238.832764   fall
                     2011-01-01  1189.914307  1228.283569  1276.126465   fall
                     2041-01-01  1209.976929  1288.678589  1329.441040   fall
                     2071-01-01  1214.395996  1276.816772  

In [13]:
dfseason45 = pd.concat(listds45)

In [14]:
dfseason45

prcptot_p10  prcptot_p50  prcptot_p90  season
lat       lon        time                                                     
66.623306 -89.045212 1981-01-01   242.776947   248.862839   253.129547  winter
                     2011-01-01   250.770477   258.080261   273.385803  winter
                     2041-01-01   268.117340   281.025696   298.878113  winter
                     2071-01-01   284.791718   298.072601   314.719666  winter
          -88.961884 1981-01-01   240.618851   246.484695   250.704636  winter
...                                      ...          ...          ...     ...
40.041039 -74.129143 2071-01-01  1217.491821  1282.380371  1355.752563    fall
          -74.045807 1981-01-01  1175.379761  1226.125488  1238.832764    fall
                     2011-01-01  1189.914307  1228.283569  1276.126465    fall
                     2041-01-01  1209.976929  1288.678589  1329.441040    fall
                     2071-01-01  1214.395996  1276.816772  1354.748291    fall

[1219296 rows x 4 columns]

# FOR Seasonal RCP85

## Loop to create all 4 dataframes

In [15]:
dsEns85= ens.create_ensemble(files85)
dsEns85.time.dt.season[0:4]
# Seasonal example 30 y means
listds85 = []
i=0
for s in dsEns85.time.dt.season[0:4]:
    print (i)
    tmp1 = dsEns85.sel(time=(dsEns85.time.dt.year>=1981))
    tmp1 = tmp1.sel(time=(tmp1.time.dt.season==s))
    window = 30
    time1 = tmp1.time[0::window]
    # 30 y means
    ds30yavgTable = tmp1.coarsen(time=window).mean()
    ds30yavgTable['time'] = time1
    # percentiles
    perc30yavgTable85 = ens.ensemble_percentiles(ds30yavgTable)
    #perc30yavgTable 
    dfSeas85 = perc30yavgTable85.drop('realization').to_dataframe().dropna()
    if i== 0: season = "winter" 
    elif i == 1: season = "spring"
    elif i == 2: season = "summer"
    elif i == 3: season = "fall"
    print (season)
    dfSeas85["season"] = season
    print(dfSeas85)
    i = i+1
    listds85.append(dfSeas85)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)



0


  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)



winter
                                 prcptot_p10  prcptot_p50  prcptot_p90  season
lat       lon        time                                                     
66.623306 -89.045212 1981-01-01   245.039719   248.596481   258.564270  winter
                     2011-01-01   260.379547   266.665070   274.814545  winter
                     2041-01-01   276.355499   296.327087   314.078552  winter
                     2071-01-01   296.454498   320.437500   370.024689  winter
          -88.961884 1981-01-01   242.724014   246.173447   256.140045  winter
...                                      ...          ...          ...     ...
40.041039 -74.129143 2071-01-01  1252.603271  1315.043823  1396.519409  winter
          -74.045807 1981-01-01  1189.647095  1231.080933  1246.812256  winter
                     2011-01-01  1202.679321  1257.817017  1306.370361  winter
                     2041-01-01  1213.784058  1263.579712  1321.775024  winter
                     2071-01-01  1245.081543 

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)



spring
                                 prcptot_p10  prcptot_p50  prcptot_p90  season
lat       lon        time                                                     
66.623306 -89.045212 1981-01-01   245.039719   248.596481   258.564270  spring
                     2011-01-01   260.379547   266.665070   274.814545  spring
                     2041-01-01   276.355499   296.327087   314.078552  spring
                     2071-01-01   296.454498   320.437500   370.024689  spring
          -88.961884 1981-01-01   242.724014   246.173447   256.140045  spring
...                                      ...          ...          ...     ...
40.041039 -74.129143 2071-01-01  1252.603271  1315.043823  1396.519409  spring
          -74.045807 1981-01-01  1189.647095  1231.080933  1246.812256  spring
                     2011-01-01  1202.679321  1257.817017  1306.370361  spring
                     2041-01-01  1213.784058  1263.579712  1321.775024  spring
                     2071-01-01  1245.081543 

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)



summer
                                 prcptot_p10  prcptot_p50  prcptot_p90  season
lat       lon        time                                                     
66.623306 -89.045212 1981-01-01   245.039719   248.596481   258.564270  summer
                     2011-01-01   260.379547   266.665070   274.814545  summer
                     2041-01-01   276.355499   296.327087   314.078552  summer
                     2071-01-01   296.454498   320.437500   370.024689  summer
          -88.961884 1981-01-01   242.724014   246.173447   256.140045  summer
...                                      ...          ...          ...     ...
40.041039 -74.129143 2071-01-01  1252.603271  1315.043823  1396.519409  summer
          -74.045807 1981-01-01  1189.647095  1231.080933  1246.812256  summer
                     2011-01-01  1202.679321  1257.817017  1306.370361  summer
                     2041-01-01  1213.784058  1263.579712  1321.775024  summer
                     2071-01-01  1245.081543 

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)



fall
                                 prcptot_p10  prcptot_p50  prcptot_p90 season
lat       lon        time                                                    
66.623306 -89.045212 1981-01-01   245.039719   248.596481   258.564270   fall
                     2011-01-01   260.379547   266.665070   274.814545   fall
                     2041-01-01   276.355499   296.327087   314.078552   fall
                     2071-01-01   296.454498   320.437500   370.024689   fall
          -88.961884 1981-01-01   242.724014   246.173447   256.140045   fall
...                                      ...          ...          ...    ...
40.041039 -74.129143 2071-01-01  1252.603271  1315.043823  1396.519409   fall
          -74.045807 1981-01-01  1189.647095  1231.080933  1246.812256   fall
                     2011-01-01  1202.679321  1257.817017  1306.370361   fall
                     2041-01-01  1213.784058  1263.579712  1321.775024   fall
                     2071-01-01  1245.081543  1309.232788  

In [16]:
dfseason85 = pd.concat(listds85)

In [17]:
dfseason85

prcptot_p10  prcptot_p50  prcptot_p90  season
lat       lon        time                                                     
66.623306 -89.045212 1981-01-01   245.039719   248.596481   258.564270  winter
                     2011-01-01   260.379547   266.665070   274.814545  winter
                     2041-01-01   276.355499   296.327087   314.078552  winter
                     2071-01-01   296.454498   320.437500   370.024689  winter
          -88.961884 1981-01-01   242.724014   246.173447   256.140045  winter
...                                      ...          ...          ...     ...
40.041039 -74.129143 2071-01-01  1252.603271  1315.043823  1396.519409    fall
          -74.045807 1981-01-01  1189.647095  1231.080933  1246.812256    fall
                     2011-01-01  1202.679321  1257.817017  1306.370361    fall
                     2041-01-01  1213.784058  1263.579712  1321.775024    fall
                     2071-01-01  1245.081543  1309.232788  1389.138306    fall

[1219296 rows x 4 columns]

## To split by region FOR RCP45 AND RCP85, Annual and seasonal

In [54]:
short_dict = {"DDE_STF_20K_REG_FOR_VUE_S": ("NM_REG_FOR", "RF"), 
            "DDE_STF_20K_UA_PER_VUE_S": ("PER_NO_UA", "UA"), 
              "DOM_BIO": ("NOM", "DB"), 
              "REG_ECO": ("NOM", "RE"), 
              "SDOM_BIO": ("NOM", "SDB"), 
              "Secteurs_Operations_Regionales": ("D_GENERAL", "SOR"), 
              "SREG_ECO": ("NOM", "SRE"),  
              "territoire_guide": ("TER_GUIDE", "TG") 
              }

for region, (name, short) in short_dict.items():
    print(region, name, short)

DDE_STF_20K_REG_FOR_VUE_S NM_REG_FOR RF
DDE_STF_20K_UA_PER_VUE_S PER_NO_UA UA
DOM_BIO NOM DB
REG_ECO NOM RE
SDOM_BIO NOM SDB
Secteurs_Operations_Regionales D_GENERAL SOR
SREG_ECO NOM SRE
territoire_guide TER_GUIDE TG


In [19]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [ ]:
 def strings_to_nan(col):
    return col.map(lambda x: np.nan if isinstance(x, str) else x)

In [20]:
d = { 'BAS-SAINT-LAURENT':"Bas-Saint-Laurent", 
     'SAGUENAY -LAC-SAINT-JEAN': "Saguenay -Lac-Saint-Jean",
     'CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES':"Capitale-Nationale-Chaudiere-Appalaches",
     'MAURICIE-CENTRE-DU-QUÉBEC':'Mauricie-Centre-du-Quebec','OUTAOUAIS':'Outaouais', 
     'ABITIBI-TEMISCAMINGUE':'Abitibi-Temiscamingue', 'COTE-NORD':'Cote-Nord',
     'NORD-DU-QUEBEC':'Nord-Du-Quebec', 'GASPESIE-ILES-DE-LA-MADELEINE':'Gaspesie-Iles-De-La-Madeleine', 
     'LANAUDIERE':'Lanaudiere','LAURENTIDES':"Laurentides", 
     "ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL":"Estrie-Monteregie-Laval-Montreal", 
     "é": "e", "É": "E", "à": "a", "è": "e", "Î": "i", "È": "E", "ô" : "o", "Ç":"C", "ç":"c",
       } 

In [101]:
for region, (name, short) in short_dict.items():
    print("------------------------------------------------------------------")
    print ("1/11... Cutting by region")
    print(region, name)
    dfregion = pd.read_feather("/home/mlopez/EXEC/Grids-polygons-regions/"+region+"lat_lon.feather")
    print ("2/11... Merging region with data")
    dfdatareg45 = pd.merge(dfregion, dfperT45.reset_index(), on=["lat","lon"])
    dfdatareg85 = pd.merge(dfregion, dfperT85.reset_index(), on=["lat","lon"])
    dfdatareg45s = pd.merge(dfregion, dfseason45.reset_index(), on=["lat","lon"])
    dfdatareg85s = pd.merge(dfregion, dfseason85.reset_index(), on=["lat","lon"])
    print ("3/11... Get mean by year for every sub-region")
    dfmean45UA = dfdatareg45.groupby(["time", name, "season"]).mean()
    dfmean45UA2 = dfmean45UA.reset_index()
    dfmean85UA = dfdatareg85.groupby(["time", name, "season"]).mean()
    dfmean85UA2 = dfmean85UA.reset_index()
    dfmean45UAs = dfdatareg45s.groupby(["time", name, "season"]).mean()
    dfmean45UA2s = dfmean45UAs.reset_index()
    dfmean85UAs = dfdatareg85s.groupby(["time", name, "season"]).mean()
    dfmean85UA2s = dfmean85UAs.reset_index()
    print ("4/11... Round RCP45")
    if (var == "tg_mean" or var== "tn_mean" or var== "tx_mean"):
        dfmean45UA2[var+"_p10"] = dfmean45UA2[var+"_p10"]- 273.15
        dfmean45UA2[var+"_p50"] = dfmean45UA2[var+"_p10"]- 273.15
        dfmean45UA2[var+"_p90"] = dfmean45UA2[var+"_p10"]- 273.15
        dfmean45UA2s[var+"_p10"] = dfmean45UA2s[var+"_p10"]- 273.15
        dfmean45UA2s[var+"_p50"] = dfmean45UA2s[var+"_p10"]- 273.15
        dfmean45UA2s[var+"_p90"] = dfmean45UA2s[var+"_p10"]- 273.15
    if (var == "solidprcptot"):
        dfmean45UA2[var+"_p10"] = dfmean45UA2[var+"_p10"]/10
        dfmean45UA2[var+"_p50"] = dfmean45UA2[var+"_p50"]/10
        dfmean45UA2[var+"_p90"] = dfmean45UA2[var+"_p90"]/10
        dfmean45UA2s[var+"_p10"] = dfmean45UA2s[var+"_p10"]/10
        dfmean45UA2s[var+"_p50"] = dfmean45UA2s[var+"_p50"]/10
        dfmean45UA2s[var+"_p90"] = dfmean45UA2s[var+"_p90"]/10
    dfmean45UA2[var+"_p10"] = round(dfmean45UA2[var+"_p10"], 2)
    dfmean45UA2[var+"_p50"] = round(dfmean45UA2[var+"_p50"], 2)
    dfmean45UA2[var+"_p90"] = round(dfmean45UA2[var+"_p90"], 2)
    dfmean45UA2s[var+"_p10"] = round(dfmean45UA2s[var+"_p10"], 2)
    dfmean45UA2s[var+"_p50"] = round(dfmean45UA2s[var+"_p50"], 2)
    dfmean45UA2s[var+"_p90"] = round(dfmean45UA2s[var+"_p90"], 2)
    print ("5/11... Round RCP85")
    if (var == "tg_mean" or var== "tn_mean" or var== "tx_mean"):
        dfmean85UA2[var+"_p10"] = dfmean85UA2[var+"_p10"]- 273.15
        dfmean85UA2[var+"_p50"] = dfmean85UA2[var+"_p10"]- 273.15
        dfmean85UA2[var+"_p90"] = dfmean85UA2[var+"_p10"]- 273.15
        dfmean85UA2s[var+"_p10"] = dfmean85UA2s[var+"_p10"]- 273.15
        dfmean85UA2s[var+"_p50"] = dfmean85UA2s[var+"_p10"]- 273.15
        dfmean85UA2s[var+"_p90"] = dfmean85UA2s[var+"_p10"]- 273.15
    if (var == "solidprcptot"):
        dfmean85UA2[var+"_p10"] = dfmean85UA2[var+"_p10"]/10
        dfmean85UA2[var+"_p50"] = dfmean85UA2[var+"_p50"]/10
        dfmean85UA2[var+"_p90"] = dfmean85UA2[var+"_p90"]/10
        dfmean85UA2s[var+"_p10"] = dfmean85UA2s[var+"_p10"]/10
        dfmean85UA2s[var+"_p50"] = dfmean85UA2s[var+"_p50"]/10
        dfmean85UA2s[var+"_p90"] = dfmean85UA2s[var+"_p90"]/10
    dfmean85UA2[var+"_p10"] = round(dfmean85UA2[var+"_p10"], 2)
    dfmean85UA2[var+"_p50"] = round(dfmean85UA2[var+"_p50"], 2)
    dfmean85UA2[var+"_p90"] = round(dfmean85UA2[var+"_p90"], 2)
    dfmean85UA2s[var+"_p10"] = round(dfmean85UA2s[var+"_p10"], 2)
    dfmean85UA2s[var+"_p50"] = round(dfmean85UA2s[var+"_p50"], 2)
    dfmean85UA2s[var+"_p90"] = round(dfmean85UA2s[var+"_p90"], 2)
    print("6/11 ... Merging columns 10 and 90")
    dfmean45UA2[var+"_p10_p90"] = dfmean45UA2[var+"_p10"].map(str)+" - "+dfmean45UA2[var+"_p90"].map(str)
    dfreg45 = dfmean45UA2[["time", name, var+"_p50", var+"_p10_p90", "season"]]
    dfmean85UA2[var+"_p10_p90"] = dfmean85UA2[var+"_p10"].map(str)+" - "+dfmean85UA2[var+"_p90"].map(str)
    dfreg85 = dfmean85UA2[["time", name, var+"_p50", var+"_p10_p90", "season"]]
    dfreg2 = pd.merge(dfreg45, dfreg85, on=["time",name, "season"], suffixes=('_45', '_85') )
    dfmean45UA2s[var+"_p10_p90"] = dfmean45UA2s[var+"_p10"].map(str)+" - "+dfmean45UA2s[var+"_p90"].map(str)
    dfreg45s =  dfmean45UA2s[["time", name, var+"_p50", var+"_p10_p90", "season"]]
    dfmean85UA2s[var+"_p10_p90"] = dfmean85UA2s[var+"_p10"].map(str)+" - "+dfmean85UA2s[var+"_p90"].map(str)
    dfreg85s =  dfmean85UA2s[["time", name, var+"_p50", var+"_p10_p90", "season"]]
    dfreg2s = pd.merge(dfreg45s, dfreg85s, on=["time",name, "season"], suffixes=('_45', '_85') )
    print("7/11 ... Merging Annual and Seasonal DFs")
    dfall = pd.concat((dfreg2, dfreg2s))
    print("8/11 ... Pivoting and ordering Table")
    dfreg3 = pd.wide_to_long(dfall,(var+"_p50_",var+"_p10_p90_"),i=["time",name,"season"],j="scenario")
    dfreg3b = dfreg3.copy()
    dfreg3b.columns = [var+"50", var+"1090"]
    dfreg4 = pd.wide_to_long(dfreg3b.reset_index(),var, i=("time",name,"season","scenario"), j="percentility").reset_index()
    for (_,df) in dfreg4.reset_index().groupby(name):
        df = df.copy()
        df["sp"] = df.season + df.percentility.map(str)
        df["Saison"] = df.time.map(str) + df.scenario.map(str)    
        df_table = pd.pivot(df,columns="Saison", index="sp", values=var)
    print ("9/11 Reindexing table")
    dftable2 = df_table.reindex(index=["annual50", "annual1090","winter50","winter1090","spring50","spring1090",
                       "summer50","summer1090","fall50","fall1090"])
    dftable2.index=["Annuel", "Percentiles (10 - 90)","Hiver","Percentiles (10 - 90)","Printemps","Percentiles (10 - 90)",
                       "Ete","Percentiles (10 - 90)","Automne","Percentiles (10 - 90)"]
    print("10/11 Getting mean for historic data")
    dftable2["1981-2010"] = ((strings_to_nan(dftable2.iloc[:,0]) + strings_to_nan(dftable2.iloc[:,1])) / 2).fillna("")
    dftable3 = dftable2.drop(columns=['1981-01-01 00:00:0045', '1981-01-01 00:00:0085', "2011-01-01 00:00:0045", "2011-01-01 00:00:0085" ])
    dftable4 = dftable3[["1981-2010", "2041-01-01 00:00:0045", "2041-01-01 00:00:0085", "2071-01-01 00:00:0045",
                         "2071-01-01 00:00:0085"]]
    dftable5 = dftable4.rename(columns={'2041-01-01 00:00:0045': 'Emissions moderees 2050',
                                        '2041-01-01 00:00:0085': 'Emissions Elevees 2050',
                                        '2071-01-01 00:00:0045': 'Emissions moderees 2080',
                                        '2071-01-01 00:00:0085': 'Emissions Elevees 2080', })
    print("11/11 ... CREATING CSV!!")
    dftable5.to_csv(name+"_"+var+"_table.csv")


------------------------------------------------------------------
1/11... Cutting by region
DDE_STF_20K_REG_FOR_VUE_S NM_REG_FOR
2/11... Merging region with data
3/11... Get mean by year for every sub-region
4/11... Round RCP45
5/11... Round RCP85
6/11 ... Merging columns 10 and 90
7/11 ... Merging Annual and Seasonal DFs
8/11 ... Pivoting and ordering Table
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
------------------------------------------------------------------
1/11... Cutting by region
DDE_STF_20K_UA_PER_VUE_S PER_NO_UA
2/11... Merging region with data
3/11... Get mean by year for every sub-region
4/11... Round RCP45
5/11... Round RCP85
6/11 ... Merging columns 10 and 90
7/11 ... Merging Annual and Seasonal DFs
8/11 ... Pivoting and ordering Table
9/11 Reindexing table
10/11 Getting mean for historic data
11/11 ... CREATING CSV!!
------------------------------------------------------------------
1/11... Cutting by region
DOM_BIO NOM
2/11.

In [25]:
dfall = pd.concat((dfreg2, dfreg2s))

In [30]:
dfall

,time,TER_GUIDE,prcptot_p50_45,prcptot_p10_p90_45,season,prcptot_p50_85,prcptot_p10_p90_85
0,1981-01-01,1a,993.74,969.76 - 1002.86,annual,990.40,968.13 - 1002.7
1,1981-01-01,2a,979.45,957.29 - 988.19,annual,971.91,959.77 - 986.03
2,1981-01-01,2b,1081.70,1063.35 - 1091.75,annual,1078.03,1053.74 - 1089.19
3,1981-01-01,2c,1161.06,1142.27 - 1179.84,annual,1162.10,1126.91 - 1170.33
4,1981-01-01,3ab,961.13,946.92 - 975.57,annual,956.57,944.79 - 971.26
...,...,...,...,...,...,...,...
395,2071-01-01,6mn,1185.44,1144.77 - 1222.4,winter,1275.90,1186.15 - 1316.01
396,2071-01-01,6opqr,1076.98,1034.46 - 1132.06,fall,1145.22,1082.04 - 1206.63
397,2071-01-01,6opqr,1076.98,1034.46 - 1132.06,spring,1145.22,1082.04 - 1206.63
398,2071-01-01,6opqr,1076.98,1034.46 - 1132.06,summer,1145.22,1082.04 - 1206.63


In [55]:
dfreg3 = pd.wide_to_long(dfall,(var+"_p50_",var+"_p10_p90_"),i=["time",name,"season"],j="scenario")
dfreg3

prcptot_p50_   prcptot_p10_p90_
time       TER_GUIDE season scenario                                 
1981-01-01 1a        annual 45              993.74   969.76 - 1002.86
                            85              990.40    968.13 - 1002.7
           2a        annual 45              979.45    957.29 - 988.19
                            85              971.91    959.77 - 986.03
           2b        annual 45             1081.70  1063.35 - 1091.75
...                                            ...                ...
2071-01-01 6opqr     spring 85             1145.22  1082.04 - 1206.63
                     summer 45             1076.98  1034.46 - 1132.06
                            85             1145.22  1082.04 - 1206.63
                     winter 45             1076.98  1034.46 - 1132.06
                            85             1145.22  1082.04 - 1206.63

[1000 rows x 2 columns]

In [61]:
dfreg3b = dfreg3.copy()
dfreg3b.columns = [var+"50", var+"1090"]
dfreg4 = pd.wide_to_long(dfreg3b.reset_index(),var, i=("time",name,"season","scenario"), j="percentility").reset_index()
dfreg4

,time,TER_GUIDE,season,scenario,percentility,prcptot
0,1981-01-01,1a,annual,45,50,993.74
1,1981-01-01,1a,annual,45,1090,969.76 - 1002.86
2,1981-01-01,1a,annual,85,50,990.4
3,1981-01-01,1a,annual,85,1090,968.13 - 1002.7
4,1981-01-01,2a,annual,45,50,979.45
...,...,...,...,...,...,...
1995,2071-01-01,6opqr,summer,85,1090,1082.04 - 1206.63
1996,2071-01-01,6opqr,winter,45,50,1076.98
1997,2071-01-01,6opqr,winter,45,1090,1034.46 - 1132.06
1998,2071-01-01,6opqr,winter,85,50,1145.22


In [98]:
for (_,df) in dfreg4.reset_index().groupby(name):
    df = df.copy()
    df["sp"] = df.season + df.percentility.map(str)
    df["Saison"] = df.time.map(str) + df.scenario.map(str)    
    df_table = pd.pivot(df,columns="Saison", index="sp", values="prcptot")
    print(df_table)

Saison     1981-01-01 00:00:0045 1981-01-01 00:00:0085 2011-01-01 00:00:0045  \
sp                                                                             
annual1090      969.76 - 1002.86       968.13 - 1002.7      998.04 - 1046.72   
annual50                  993.74                 990.4               1022.01   
fall1090        969.76 - 1002.86       968.13 - 1002.7      998.04 - 1046.72   
fall50                    993.74                 990.4               1022.01   
spring1090      969.76 - 1002.86       968.13 - 1002.7      998.04 - 1046.72   
spring50                  993.74                 990.4               1022.01   
summer1090      969.76 - 1002.86       968.13 - 1002.7      998.04 - 1046.72   
summer50                  993.74                 990.4               1022.01   
winter1090      969.76 - 1002.86       968.13 - 1002.7      998.04 - 1046.72   
winter50                  993.74                 990.4               1022.01   

Saison     2011-01-01 00:00:0085 2041-0

Saison     1981-01-01 00:00:0045 1981-01-01 00:00:0085 2011-01-01 00:00:0045  \
sp                                                                             
annual1090     1089.29 - 1133.02     1081.25 - 1130.81     1119.64 - 1203.14   
annual50                 1116.71               1118.12                1147.4   
fall1090       1089.29 - 1133.02     1081.25 - 1130.81     1119.64 - 1203.14   
fall50                   1116.71               1118.12                1147.4   
spring1090     1089.29 - 1133.02     1081.25 - 1130.81     1119.64 - 1203.14   
spring50                 1116.71               1118.12                1147.4   
summer1090     1089.29 - 1133.02     1081.25 - 1130.81     1119.64 - 1203.14   
summer50                 1116.71               1118.12                1147.4   
winter1090     1089.29 - 1133.02     1081.25 - 1130.81     1119.64 - 1203.14   
winter50                 1116.71               1118.12                1147.4   

Saison     2011-01-01 00:00:0085 2041-0

In [63]:
df_table

Saison,1981-01-01 00:00:0045,1981-01-01 00:00:0085,2011-01-01 00:00:0045,2011-01-01 00:00:0085,2041-01-01 00:00:0045,2041-01-01 00:00:0085,2071-01-01 00:00:0045,2071-01-01 00:00:0085
sp,,,,,,,,
annual1090,952.85 - 986.1,956.23 - 991.36,975.43 - 1071.48,994.07 - 1056.49,994.94 - 1087.33,1030.27 - 1153.61,1034.46 - 1132.06,1082.04 - 1206.63
annual50,972.13,967.2,1014.68,1024.58,1045.23,1093.05,1076.98,1145.22
fall1090,952.85 - 986.1,956.23 - 991.36,975.43 - 1071.48,994.07 - 1056.49,994.94 - 1087.33,1030.27 - 1153.61,1034.46 - 1132.06,1082.04 - 1206.63
fall50,972.13,967.2,1014.68,1024.58,1045.23,1093.05,1076.98,1145.22
spring1090,952.85 - 986.1,956.23 - 991.36,975.43 - 1071.48,994.07 - 1056.49,994.94 - 1087.33,1030.27 - 1153.61,1034.46 - 1132.06,1082.04 - 1206.63
spring50,972.13,967.2,1014.68,1024.58,1045.23,1093.05,1076.98,1145.22
summer1090,952.85 - 986.1,956.23 - 991.36,975.43 - 1071.48,994.07 - 1056.49,994.94 - 1087.33,1030.27 - 1153.61,1034.46 - 1132.06,1082.04 - 1206.63
summer50,972.13,967.2,1014.68,1024.58,1045.23,1093.05,1076.98,1145.22
winter1090,952.85 - 986.1,956.23 - 991.36,975.43 - 1071.48,994.07 - 1056.49,994.94 - 1087.33,1030.27 - 1153.61,1034.46 - 1132.06,1082.04 - 1206.63


In [87]:
dftable2 = df_table.reindex(index=["annual50", "annual1090","winter50","winter1090","spring50","spring1090",
                       "summer50","summer1090","fall50","fall1090"])

In [88]:
dftable2.index=["Annuel", "Percentiles (10 - 90)","Hiver","Percentiles (10 - 90)","Printemps","Percentiles (10 - 90)",
                       "Ete","Percentiles (10 - 90)","Automne","Percentiles (10 - 90)"]

In [90]:
def strings_to_nan(col):
    return col.map(lambda x: np.nan if isinstance(x, str) else x)

dftable2["1981-2010"] = ((strings_to_nan(dftable2.iloc[:,0]) + strings_to_nan(dftable2.iloc[:,1])) / 2).fillna("")
dftable2

dftable3 = dftable2.drop(columns=['1981-01-01 00:00:0045', '1981-01-01 00:00:0085', "2011-01-01 00:00:0045", "2011-01-01 00:00:0085" ])
dftable3

dftable4 = dftable3[["1981-2010", "2041-01-01 00:00:0045", "2041-01-01 00:00:0085", "2071-01-01 00:00:0045",
                     "2071-01-01 00:00:0085"]]
dftable4

dftable5 = dftable4.rename(columns={'2041-01-01 00:00:0045': 'Emissions moderees 2050',
                                    '2041-01-01 00:00:0085': 'Emissions Elevees 2050',
                                    '2071-01-01 00:00:0045': 'Emissions moderees 2080',
                                    '2071-01-01 00:00:0085': 'Emissions Elevees 2080', })

In [96]:
dftable5

Saison,1981-2010,Emissions moderees 2050,Emissions Elevees 2050,Emissions moderees 2080,Emissions Elevees 2080
Annuel,969.665,1045.23,1093.05,1076.98,1145.22
Percentiles (10 - 90),,994.94 - 1087.33,1030.27 - 1153.61,1034.46 - 1132.06,1082.04 - 1206.63
Hiver,969.665,1045.23,1093.05,1076.98,1145.22
Percentiles (10 - 90),,994.94 - 1087.33,1030.27 - 1153.61,1034.46 - 1132.06,1082.04 - 1206.63
Printemps,969.665,1045.23,1093.05,1076.98,1145.22
Percentiles (10 - 90),,994.94 - 1087.33,1030.27 - 1153.61,1034.46 - 1132.06,1082.04 - 1206.63
Ete,969.665,1045.23,1093.05,1076.98,1145.22
Percentiles (10 - 90),,994.94 - 1087.33,1030.27 - 1153.61,1034.46 - 1132.06,1082.04 - 1206.63
Automne,969.665,1045.23,1093.05,1076.98,1145.22
Percentiles (10 - 90),,994.94 - 1087.33,1030.27 - 1153.61,1034.46 - 1132.06,1082.04 - 1206.63


In [97]:
dftable5.to_csv(region+"_"var"_table.csv")